In [1]:
import os
import re
import csv

In [2]:
path = 'C:\\Users\\lm709\\OneDrive\\codes\\IR_lab\\mini_newsgroups_lab2'
index = {}
ans = {}

In [3]:
for dirpath, dirnames, files in os.walk(path, topdown=True):
        for file in files:
            fr = open(dirpath+'\\'+file, encoding='utf-8', errors='ignore')
            count = 0
            for line in fr.readlines():
                if(line[0:7] == "Lines: "):
                    for n in re.findall(r"\d+", line):
                        count = int(n)
            fr = open(dirpath+'\\'+file, encoding='utf-8', errors='ignore')
            body = fr.readlines()[-count:]
            fw = open(dirpath+'\\'+file, 'a')
            fw.seek(0)
            fw.truncate()
            for line in body:
                fw.write(line)
            fw.close()
            fr.close()

In [4]:
cnt = 1
for dirpath, dirnames, files in os.walk(path, topdown=True):
    for file in files:
        os.rename(os.path.join(dirpath+'', str(file)), os.path.join(dirpath+'', str(cnt)))
        cnt = cnt + 1

In [5]:
from nltk.tokenize import RegexpTokenizer
for dirpath, dirnames, files in os.walk(path, topdown=True):
    for file in files:
        fr = open(dirpath+'\\'+file, errors='ignore')
        word_tokenizer = RegexpTokenizer('[A-Za-z]+')
        terms = word_tokenizer.tokenize(fr.read())
        terms.sort()
        for word in terms:
            word = word.lower()
            if word in index:
                index[word].add(file)
            else:
                index[word] = set([file])
index = sorted(index.items(), key = lambda x:x[0])  

In [6]:
for word, doc_id in index:
        temp = [int(x) for x in doc_id]
        temp.sort()
        ans[word] = temp
with open('Inversed index table.csv', 'w') as csvfile:
    w = csv.writer(csvfile)
    for word in ans:
        new_list = []
        new_list.append(word)
        new_list.append(str(len(ans[word])))
        for element in ans[word]:
            new_list.append(str(element))
        w.writerow(new_list)

In [7]:
index = dict(index)
pattern = re.compile(r'(^(\S+)(\s+)?(AND|OR)(\s+)?(\S+)?$)*')
while True:
    print("*****************************************************************\n")
    print("*****************************************************************\n")
    print("**********************Enter exit to quit*************************")
    query = input('Please enter the content of query (or press enter to quit): ')
    if query.lower() == 'exit':
        break
    else:
        if re.match(pattern, query):
            keywords = re.split(r'\s+',query)
            idx = index.get(keywords[0], {})
            i = 1
            while i <= (len(keywords)-1)/2:
                if keywords[i] == 'AND':
                    idx = idx & index.get(keywords[i+1], set())
                elif keywords[i] == 'OR':
                    idx = idx | index.get(keywords[i+1], set())
                elif keywords[i] == 'NOT':
                    idx = idx & ~index.get(keywords[i+1], set())
                i = i + 1
            if len(idx) == 0:
                print('No results retrieved.')
            else:
                print('The result is: ')
                for id in idx:
                    print(id + ' ')
        else:
            print('Invalid input!')

*****************************************************************

*****************************************************************

**********************Enter exit to quit*************************
